# Entanglement forging

In this tutorial, we show how to simulate entanglement forging for a water molecule at equilibrium.

In [ ]:
import pyscf
import pyscf.mcscf
import ffsim
import math
import numpy as np

# Build a water molecule
radius_1 = 0.958  # position for the first H atom
radius_2 = 0.958  # position for the second H atom
bond_angle_deg = 104.478  # bond angles.

h1_x = radius_1
h2_x = radius_2 * math.cos(math.pi / 180 * bond_angle_deg)
h2_y = radius_2 * math.sin(math.pi / 180 * bond_angle_deg)

mol = pyscf.gto.Mole()
mol.build(
    atom=[
        ["O", (0, 0, 0)],
        ["H", (h1_x, 0, 0)],
        ["H", (h2_x, h2_y, 0)],
    ],
    basis="sto-6g",
    symmetry="c2v",
)
hartree_fock = pyscf.scf.RHF(mol)
hartree_fock.kernel()

# Define active space
active_space = [1, 2, 4, 5, 6]
norb = len(active_space)
n_electrons = int(sum(hartree_fock.mo_occ[active_space]))
n_alpha = (n_electrons + hartree_fock.mol.spin) // 2
n_beta = (n_electrons - hartree_fock.mol.spin) // 2
nelec = (n_alpha, n_beta)

# Compute FCI energy
cas = pyscf.mcscf.CASCI(hartree_fock, ncas=len(active_space), nelecas=nelec)
mo = cas.sort_mo(active_space, base=0)
energy_fci = cas.kernel(mo)[0]

# Get molecular data and molecular Hamiltonian (one- and two-body tensors)
mol_data = ffsim.MolecularData.from_hartree_fock(
    hartree_fock, active_space=active_space
)
mol_hamiltonian = mol_data.hamiltonian

In [ ]:
import numpy as np

n_reps = 1

# Construct ansatz operator
interaction_pairs = [(0, 1), (3, 4), (1, 4), (0, 2), (3, 4)]
thetas = np.zeros(n_reps * len(interaction_pairs))
operator = ffsim.HopGateAnsatzOperator(interaction_pairs, thetas=thetas)

# Construct ansatz state
reference_occupations_spatial = [(0, 1, 2), (1, 2, 3), (1, 2, 4)]
reference_occupations = list(
    zip(reference_occupations_spatial, reference_occupations_spatial)
)
hamiltonian = ffsim.linear_operator(mol_hamiltonian, norb=norb, nelec=nelec)
ansatz_state = ffsim.multireference_state(
    hamiltonian, operator, reference_occupations, norb=norb, nelec=nelec
)

# Compute the energy ⟨ψ|H|ψ⟩ of the ansatz state
energy = np.real(np.vdot(ansatz_state, hamiltonian @ ansatz_state))
print(f"Energy at initialialization: {energy}")

In [ ]:
import scipy.optimize


def fun(x):
    # Initialize the ansatz operator from the parameter vector
    operator = ffsim.HopGateAnsatzOperator(interaction_pairs, x)
    # Compute ansatz state
    ansatz_state = ffsim.multireference_state(
        hamiltonian, operator, reference_occupations, norb=norb, nelec=nelec
    )
    # Compute the energy ⟨ψ|H|ψ⟩ of the ansatz state
    return np.real(np.vdot(ansatz_state, hamiltonian @ ansatz_state))


result = scipy.optimize.minimize(
    fun, x0=operator.thetas, method="COBYLA", options=dict(maxiter=100)
)

print(f"Number of parameters: {len(result.x)}")
print(result)